In [4]:
import pandas as pd
import numpy as np


df1 = pd.read_excel("dummy1.xlsx");
df2 = pd.read_excel("dummy2.xlsx");

df1

,ID,Caregiver_for_Veteran,Other_caregiver,Veteran,Other,Age,Sex,Time_Providing_Care,Race,Use_of_Video_Conferencing,...,Caregivers_Portal_Video_Use,Family_members__Portal_Video_Us,Friends_Portal_Video_Use,Others_Portal_Video_Use,Not_using_Portal_video_Portal_Video_Use,Portal_Chronic_Disease_Management_Help,SMS_text_Connection_Technology_Use,Phone_Connection_Technology_Use,Video_Portal_Connection_Technology_Use,Other_Connection_Technology_Use
0,1,0,0,0,1,0-17,F,2-3years,American Indian or Alaska Native,3-6months,...,0,0,0,3,0,0,1,0,1,1
1,2,1,1,1,0,65+,M,1-2years,American Indian or Alaska Native,3-6months,...,0,0,1,2,1,0,1,0,0,1
2,3,0,0,1,1,18-25,F,3-4years,Black or African American,3-6months,...,0,1,1,2,0,0,0,0,0,0
3,4,1,1,1,0,0-17,M,2-3years,White,9-12months,...,1,1,0,3,1,0,1,0,0,0
4,5,0,0,0,0,65+,M,2-3years,White,3-6months,...,0,1,1,1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,1,0,1,0,0-17,M,2-3years,Asian,3-6months,...,0,0,0,1,0,1,0,0,0,0
496,497,0,0,0,0,0-17,F,3-4years,Black or African American,9-12months,...,0,0,0,1,0,1,0,0,0,1
497,498,0,0,0,0,0-17,F,2-3years,Asian,9-12months,...,1,0,1,2,1,0,0,0,0,1
498,499,1,1,0,0,25-34,F,1-2years,Black or African American,3-6months,...,0,1,1,2,1,0,1,1,1,1


In [16]:
dropped1 = df1.drop(columns=['ID', 'Age', 'Sex', 'Time_Providing_Care', 'Race', 'Caregiver_for_Veteran', 'Other_caregiver','Veteran','Other',
                'Use_of_Video_Conferencing', 'Hypertension_high_blood_pressure_Chronic_Conditions', 'Hyperlipidemia_high_blood_cholesterol_or_triglyceride_levels_Chronic_Conditions', 
                 'Allergies', 'sinusitis_and_other_upper_respiratory_conditions_Chronic_Conditions', 'Arthritis_Chronic_Conditions', 'Mood_Disorder_depression_and_bipolar_disorder_Chronic_Conditions', 
                'Diabetes_Type1_and_Type_2_Chronic_Conditions', 'Anxiety_Disorder_Chronic_Conditions', 'Asthma_Chronic_Conditions', 'Coronary_artery_disease_includes_myocardial_infarction_heart_attack_Chronic_Conditions', 
                'Thyroid_disorder_Chronic_Conditions', 'Chronic_obstructive_lung_disease_and_bronchiectasis_Chronic_Condition', 'Traumatic_Brain_Injury__TBI_Chronic_Conditions', 'Post_Traumatic_Stress_Disorder_PTSD_Chronic_Conditions', 
                'None_of_the_above__Chronic_Conditions', 'Caregivers_Portal_Video_Use', 'Family_members__Portal_Video_Us', 'Friends_Portal_Video_Use', 'Others_Portal_Video_Use', 'Not_using_Portal_video_Portal_Video_Use', 'Portal_Chronic_Disease_Management_Help', 
                'SMS_text_Connection_Technology_Use', 'Phone_Connection_Technology_Use', 'Video_Portal_Connection_Technology_Use', 'Other_Connection_Technology_Use'])

dropped2 = df2.drop(columns=['ID', 'Age', 'Sex', 'Time_Providing_Care', 'Race', 'Caregiver_for_Veteran', 'Other_caregiver','Veteran','Other',
                'Use_of_Video_Conferencing', 'Hypertension_high_blood_pressure_Chronic_Conditions', 'Hyperlipidemia_high_blood_cholesterol_or_triglyceride_levels_Chronic_Conditions', 
                 'Allergies', 'sinusitis_and_other_upper_respiratory_conditions_Chronic_Conditions', 'Arthritis_Chronic_Conditions', 'Mood_Disorder_depression_and_bipolar_disorder_Chronic_Conditions', 
                'Diabetes_Type1_and_Type_2_Chronic_Conditions', 'Anxiety_Disorder_Chronic_Conditions', 'Asthma_Chronic_Conditions', 'Coronary_artery_disease_includes_myocardial_infarction_heart_attack_Chronic_Conditions', 
                'Thyroid_disorder_Chronic_Conditions', 'Chronic_obstructive_lung_disease_and_bronchiectasis_Chronic_Condition', 'Traumatic_Brain_Injury__TBI_Chronic_Conditions', 'Post_Traumatic_Stress_Disorder_PTSD_Chronic_Conditions', 
                'None_of_the_above__Chronic_Conditions', 'Caregivers_Portal_Video_Use', 'Family_members__Portal_Video_Us', 'Friends_Portal_Video_Use', 'Others_Portal_Video_Use', 'Not_using_Portal_video_Portal_Video_Use', 'Portal_Chronic_Disease_Management_Help', 
                'SMS_text_Connection_Technology_Use', 'Phone_Connection_Technology_Use', 'Video_Portal_Connection_Technology_Use', 'Other_Connection_Technology_Use'])

In [17]:
encoding = {
    'Agree': 1,
    'StronglyAgree': 2,
    'Strongly Agree': 2,
    'StronglyDisagree': -2,
    'Strongly Disagree': -2,
    'Disagree': -1,
    'Neutral': 0,
    'Yes': 1,
    'No': -1,
    'Rarely':-1,
    'Never':-2,
    'Sometimes':1,
    'Always':2,
    '0':0,
    '1':1
}
def f(s):
    l = []
    for b in s:
        #print(b)
        #print(type(b))
        if (b in encoding):
            l.append(encoding[b])
        else:
            l.append(b)
    return pd.Series(l)
    #print('\n')

dropped1

,Motivation,Physical_Exhaustion,Life_Satisfaction,Relationship_Impact,Concern_About_Future,Competing_Demand,Demands_Strength,Health_Change,Financial_Decrease,Same_Self,...,Feel_Excluded,Difficult_to_make_friends,Need_Company,Unable_To_Communicate,Feel_Completely_Alone,Waiting_for_Contact,No_One_Understands_Me,Cant_Tolerate_Being_Alone,No_One_To_Talk_To,Unhappy_Alone
0,0,0,Neutral,StronglyAgree,Agree,StronglyAgree,StronglyAgree,Agree,StronglyDisagree,Agree,...,Always,Always,Never,Rarely,Sometimes,Sometimes,Never,Always,Never,0
1,1,0,StronglyDisagree,Agree,StronglyDisagree,StronglyAgree,StronglyDisagree,StronglyAgree,StronglyAgree,StronglyAgree,...,Sometimes,Never,Never,Rarely,Sometimes,Always,Rarely,Rarely,Always,0
2,0,0,Neutral,StronglyAgree,Agree,StronglyAgree,Disagree,Neutral,StronglyAgree,Disagree,...,Always,Sometimes,Always,Always,Sometimes,Never,Sometimes,Always,Sometimes,0
3,0,1,Agree,StronglyDisagree,StronglyDisagree,StronglyAgree,StronglyDisagree,StronglyAgree,Disagree,StronglyAgree,...,Rarely,Always,Sometimes,Rarely,Never,Always,Never,Sometimes,Never,1
4,1,1,Disagree,Disagree,StronglyAgree,Agree,StronglyDisagree,Neutral,Neutral,Neutral,...,Always,Rarely,Rarely,Rarely,Always,Rarely,Sometimes,Always,Never,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,Disagree,Disagree,Neutral,StronglyDisagree,StronglyDisagree,Agree,Agree,Neutral,...,Sometimes,Always,Sometimes,Sometimes,Always,Never,Rarely,Rarely,Rarely,1
496,0,0,Agree,Neutral,Agree,Neutral,Disagree,Neutral,StronglyAgree,StronglyDisagree,...,Always,Rarely,Rarely,Never,Sometimes,Never,Always,Sometimes,Never,1
497,0,0,Agree,Agree,StronglyDisagree,Disagree,StronglyAgree,Neutral,StronglyDisagree,Agree,...,Sometimes,Sometimes,Sometimes,Never,Never,Rarely,Rarely,Sometimes,Always,0
498,0,1,Disagree,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,...,Always,Rarely,Rarely,Rarely,Rarely,Rarely,Sometimes,Never,Sometimes,1


In [19]:
data1 = dropped1.apply(f, axis = 0);
data2 = dropped2.apply(f, axis = 0);

data1
data2

,Motivation,Physical_Exhaustion,Life_Satisfaction,Relationship_Impact,Concern_About_Future,Competing_Demand,Demands_Strength,Health_Change,Financial_Decrease,Same_Self,...,Feel_Excluded,Difficult_to_make_friends,Need_Company,Unable_To_Communicate,Feel_Completely_Alone,Waiting_for_Contact,No_One_Understands_Me,Cant_Tolerate_Being_Alone,No_One_To_Talk_To,Unhappy_Alone
0,1,1,2,-1,0,1,-1,0,2,2,...,-1,2,1,-1,-1,1,-1,-2,-1,0
1,1,0,-1,0,2,2,-2,0,2,2,...,-2,-1,2,2,-2,2,-2,-2,1,1
2,0,1,-2,-1,-2,0,2,0,-1,0,...,-2,2,2,-1,-1,2,2,-1,-1,0
3,1,1,-2,2,-1,2,2,2,0,1,...,-2,2,1,1,1,1,1,-2,2,1
4,1,0,2,1,-1,-1,1,1,1,-1,...,1,-2,-2,2,2,-2,-1,1,-2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1,0,0,0,2,1,2,1,-1,1,...,-1,1,-1,-1,1,-1,-2,2,1,0
496,0,0,0,-2,-1,0,1,-2,1,-2,...,-1,2,-2,-1,2,1,-1,-2,-1,0
497,0,1,1,2,-1,-2,2,-1,-2,2,...,-1,2,-2,-1,-1,-1,-1,-1,-2,1
498,0,1,1,2,2,0,-2,1,0,-1,...,-1,-2,2,1,1,2,1,1,-2,1
